## Package

In [ ]:
import numpy as np
import os
import argparse
import torch
import torchvision
import torchvision.transforms as transforms

## Config

In [ ]:
parser = {
    "root": "../",
    "seeds": 123,
    "batch_size": 16,
    "class_a_index": 5, # 5 -> dog
    "class_b_index": 3, # 3 -> cat
    "class_a_size": None, # Use None to read all class_a samples (5000 rows)
    "class_b_size": None,
    "epoch": 400,
    "download_cifar10": False, # set to True if you have not downloaded cifar10 dataset 
}
config = argparse.Namespace(**parser)

## Reprodicibility

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    
set_seed(config.seeds)

## Load Data

In [ ]:
class CustomDataLoader:
    
    def __init__(self, class_a_size=None, class_b_size=None, seeds=123, download=True):
        """
        Arguments:
            class_a_size: The number of samples for class a (e.g. dog).
            class_b_size: The number of samples for class b (e.g. cat).
        """
        self.class_a_size = class_a_size # e.g. 5000
        self.class_b_size = class_b_size # e.g. 5000
        self.seeds = seeds
        self.classes = ['plane', 'car', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
        self.download = download
        
    def sampling(self, indices, num, seeds=123):
        """
        Sample num samples from indices randomly. If num is a float and 0.0 <= num <= 1.0,
        we sample num percentage of samples from indices.
        """
        np.random.seed(self.seeds)
        if isinstance(num, float) and 0 <= num <= 1:
            size = int(num * len(indices))
            samples = np.random.choice(indices, size)
        elif isinstance(num, int) and num <= len(indices):
            size = num
            samples = np.random.choice(indices, size)
        elif num is None:
            samples = indices
        else:
            print("Please make sure 'num' is in the correct range")
        return samples       
        
    def get_subset_index(self, labels, targets):
        
        # get subsets
        sample_indices = []
        for i in range(len(labels)):
            if labels[i] in targets:
                sample_indices.append(i)
            
        return sample_indices

    def load_dataset(self, data_dir, batch_size, train_a_label=5, train_b_label=3):
        
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        
        # 1. read train data belonged to train_labels
        trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True,
                                                download=self.download, transform=transform)
        
        # 1-1. get index belonged to class a and b with sampling
        class_a_indices = self.get_subset_index(trainset.targets, [train_a_label])
        class_a_indices = self.sampling(class_a_indices, self.class_a_size, self.seeds)
        
        class_b_indices = self.get_subset_index(trainset.targets, [train_b_label])
        class_b_indices = self.sampling(class_b_indices, self.class_b_size, self.seeds)
        
        train_index_subset = np.concatenate((class_a_indices, class_b_indices))
        trainsubset = torch.utils.data.Subset(trainset, train_index_subset)

        # 1. read test data and separate them into (dog, cat) and (the other 8)
        testset = torchvision.datasets.CIFAR10(root=data_dir, train=False,
                                               download=self.download, transform=transform)
        test_index_class_ab = self.get_subset_index(
            labels=testset.targets,
            targets=[train_a_label, train_b_label]
        )
        test_index_others = self.get_subset_index(
            labels=testset.targets,
            targets=[cls for cls in range(len(self.classes)) if cls not in [train_a_label, train_b_label]]
        )
        testsubset_ab = torch.utils.data.Subset(testset, test_index_class_ab)
        testsubset_others = torch.utils.data.Subset(testset, test_index_others)

        return trainsubset, (testsubset_ab, testsubset_others)

    def generate_loader(self, dataset, batch_size, shuffle=True, drop_last=True):
        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True
        )
        return dataloader

In [ ]:
mydataloader = CustomDataLoader(
    class_a_size=config.class_a_size,
    class_b_size=config.class_b_size,
    seeds=config.seeds,
    download=config.download_cifar10)

trainset, (testset_ab, testset_others) = mydataloader.load_dataset(
    data_dir=os.path.join(config.root, "data"),
    batch_size=config.batch_size,
    train_a_label=config.class_a_index, # dog
    train_b_label=config.class_b_index, # cat
)

train_loader = mydataloader.generate_loader(
    dataset=trainset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True,
)

test_ab_loader = mydataloader.generate_loader(
    dataset=testset_ab,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

test_others_loader = mydataloader.generate_loader(
    dataset=testset_others,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=False,
)

## Model Architecture

### CNN Based

In [ ]:
class CNNBased(nn.Module):
    def __init__(self, ):
        super(CNNBased, self).__init__()
        
    def forward(self, inputs):
        pass

### ResNet

In [ ]:
class CustomResNet(nn.Module):
    def __init__(self, ):
        super(CustomResNet, self).__init__()
        
    def forward(self, inputs):
        pass

## Train Test Functions

### Train

In [ ]:
def train(trainloader, criterion, optimizer):
    pass

### Test

In [ ]:
def evaluation(testloader):
    pass

### Utils

- Save anything for plots?

## Start Training

### Reload Data

### Training Requirements

- optimizer
- criterion

### Start Here